# |In this notebook we fit a SEIR model to the Moscow Covid-19 data

In [140]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.integrate import odeint
import lmfit
from tqdm.auto import tqdm

In [141]:
sns.set()
%matplotlib inline

In [142]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
%autoreload 2

In [144]:
from sir_models.seir import SEIR, DayAheadFitter, CurveFitter
from sir_models.utils import stepwise

# Load data

In [145]:
df = pd.read_csv('data/moscow_prepared.csv', parse_dates=['date'])

# Model

In [146]:
train_subset = df[(df.date >= '2020-03-25') & (df.date <= '2020-11-30')]

In [147]:
test_subset = df[df.date > train_subset.iloc[-1].date]
test_subset.date[:3]

264   2020-12-01
265   2020-12-02
266   2020-12-03
Name: date, dtype: datetime64[ns]

In [ ]:
model = SEIR()
fitter = CurveFitter()
fitter.fit(model, train_subset)

MAE: 7686.145908875428
MAE: 7686.145908875428
MAE: 7686.145908875428
MAE: 7686.1462805682995
MAE: 7686.145858858899
MAE: 7686.145852503804
MAE: 7686.145912519189
MAE: 7686.145908702147
MAE: 7686.145907882512
MAE: 23967.02043662139
MAE: 29522.980255354523
MAE: 3182.8434143797367
MAE: 3182.843400648422
MAE: 3182.8434112678715
MAE: 3182.843419534659
MAE: 3182.8434100164086
MAE: 3182.8434262800934
MAE: 3182.8434184236276
MAE: 2143.7338235828847
MAE: 2143.733814839269
MAE: 2143.7336734371906
MAE: 2143.733704352233
MAE: 2143.733669396816
MAE: 2143.7338196841542
MAE: 2143.7338261512878
MAE: 24654.9921518736
MAE: 2095.430081698885
MAE: 2095.4300729213514
MAE: 2095.430080060209
MAE: 2095.4300765295643
MAE: 2095.43007712309
MAE: 2095.430079612773
MAE: 2095.4300790051884
MAE: 2259.217662931757
MAE: 2079.393545544749
MAE: 2079.3935485933043
MAE: 2079.393544673625
MAE: 2079.3935445436923
MAE: 2079.393554256439
MAE: 2079.393544844113
MAE: 2079.3935446904757
MAE: 2079.81024756905
MAE: 2079.8102577548

In [ ]:
result = fitter.result
result

In [ ]:
plt.figure()
history.rt.plot()
history.beta.plot()
plt.legend()
plt.show()

In [ ]:
train_initial_conditions = model.get_initial_conditions(train_subset)
train_t = np.arange(len(train_subset))

(S, E, I, R, D), history = model.predict(train_t, train_initial_conditions)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, I, label='infected')
plt.plot(train_subset.date, E, label='exposed')
plt.plot(train_subset.date, D, label='dead')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(history.new_infected, label='daily infected')
plt.plot(history.new_dead, label='daily deaths')
plt.plot(history.new_recovered, label='daily recovered')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_recovered'], label='ground truth')
plt.plot(train_subset.date, R, label='predicted')
plt.legend()
plt.title('Recovered')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_dead'], label='ground truth')
plt.plot(train_subset.date, D, label='predicted')
plt.legend()
plt.title('Total deaths')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_recovered'], label='ground truth')
plt.plot(train_subset.date, R, label='predicted')
plt.legend()
plt.title('Total recovered')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['infected'], label='ground truth')
plt.plot(train_subset.date, I, label='predicted')
plt.legend()
plt.title('Active cases')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.arange(len(train_subset)), train_subset['infected_per_day'], label='ground truth')
plt.plot(history.new_infected, label='predicted')
plt.legend()
plt.title('Daily new cases')
plt.show()

# Obtain forecast

In [ ]:
test_t = len(train_subset) + np.arange(len(test_subset))

In [ ]:
train_t, test_t

In [ ]:
test_initial_conds = (S[-1], E[-1], I[-1], R[-1], D[-1])

In [ ]:
(test_S, test_E, test_I, test_R, test_D), history = model.predict(test_t, test_initial_conds)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['total_dead'], label='train ground truth')
plt.plot(train_subset.date, D, label='train fit')

plt.plot(test_subset.date, test_subset['total_dead'], label='test ground truth', color='red')
plt.plot(test_subset.date, test_D, label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Total deaths')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['infected'], label='train ground truth')
plt.plot(train_subset.date, I, label='train fit')

plt.plot(test_subset.date, test_subset['infected'], label='test ground truth', color='red')
plt.plot(test_subset.date, test_I, label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Active cases')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['infected'].cumsum(), label='train ground truth')
plt.plot(train_subset.date, I.cumsum(), label='train fit')

plt.plot(test_subset.date, train_subset['infected'].sum()+test_subset['infected'].cumsum(), label='test ground truth', color='red')
plt.plot(test_subset.date, I.sum()+test_I.cumsum(), label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Cumulative Infected')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_subset.date, train_subset['infected_per_day'].cumsum(), label='train ground truth')
plt.plot(train_subset.date, (pd.Series(I).diff() + pd.Series(R).diff() + pd.Series(D).diff()).cumsum(), label='train fit')

plt.plot(test_subset.date, train_subset['infected_per_day'].sum()+test_subset['infected_per_day'].cumsum(), label='test ground truth', color='red')
plt.plot(test_subset.date,  (pd.Series(I).diff() + pd.Series(R).diff() + pd.Series(D).diff()).sum()+(pd.Series(test_I).diff() + pd.Series(test_R).diff() + pd.Series(test_D).diff()).cumsum(), label='test forecasted', color='red', linestyle=':')
plt.legend()
plt.title('Cumulative new cases')
plt.show()

# 1-day ahead evaluate

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
n_eval_points = 10
eval_period_start = '2020-05-01'
eval_df = df[df.date >= eval_period_start].sample(n_eval_points)
pred_dates = []

true_D, true_R = [], []

baseline_pred_D, baseline_pred_R = [], []
model_pred_D, model_pred_R = [], []

for row in tqdm(eval_df.itertuples(), total=len(eval_df)):
    train_df = df.iloc[:row.Index]
    
    pred_dates.append(row.date)
    prev_day = train_df.iloc[-1]
    pred_D = prev_day.total_dead
    pred_R = prev_day.total_recovered
    
    model = SEIR(verbose=False)
    fitter = DayAheadFitter()
    fitter.fit(model, train_df)
    
    train_initial_conditions = model.get_initial_conditions(train_df)
    train_t = np.arange(len(train_df))
    (S, E, I, R, D), history = model.predict(train_t, train_initial_conditions)
    
    test_initial_conds = (S[-1], E[-1], I[-1], R[-1], D[-1])
    
    (S, E, I, R, D), history = model.predict([train_t[-1], train_t[-1]+1], test_initial_conds)
    
    
    model_pred_D.append(D[-1])
    model_pred_R.append(R[-1])
    baseline_pred_D.append(pred_D)
    baseline_pred_R.append(pred_R)
    true_D.append(row.total_dead)
    true_R.append(row.total_recovered)



In [ ]:
baseline_D_mae = mean_absolute_error(true_D, baseline_pred_D)
baseline_R_mae = mean_absolute_error(true_R, baseline_pred_R)

model_D_mae = mean_absolute_error(true_D, model_pred_D)
model_R_mae = mean_absolute_error(true_R, model_pred_R)

print('Baseline D mae', round(baseline_D_mae, 3))
print('Model D mae', round(model_D_mae, 3))

print('Baseline R mae', round(baseline_R_mae, 3))
print('Model R mae', round(model_R_mae, 3))